In [88]:
!pip install plotly
# !pip install plotly==5.4.0
!pip install flaml
# !pip install FLAML==0.7.1
!pip install ppscore
# !pip install scikit-learn==0.20.2
!pip install -U scikit-learn

  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1


  Using cached scikit_learn-1.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ppscore 1.2.0 requires scikit-learn<1.0.0,>=0.20.2, but you have scikit-learn 1.0.1 which is incompatible.


In [124]:
import pandas as pd  # Import Panda Library
import numpy as np # Import Numpy Library

import plotly.io as pio
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

pd.options.plotting.backend = "plotly"
pio.templates.default = 'seaborn'

# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

from flaml import AutoML

import xgboost as xgb

import ppscore as pps

import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [90]:
data_df = pd.read_excel('/content/drive/My Drive/data/BCJ.xlsx') # Assigning data_df variable of our excel sheet to panda  data framework

In [91]:
data_df.drop(0, inplace=True) # Removing the serial number column from our data set

In [92]:
data_df.drop('SI. No.', axis=1, inplace=True)  # Removing the serial number column from our data set

In [93]:
data_df.columns = ['b_j', 'h_c', 'f_c', 'T_Ast', 'B_Ast', 'L', 'f_yb', 'phi_s', 'P_y', 'x_b', 'h_b_by_h_c', 'tau'] # Labelling input and output variables in column vector.

In [94]:
data_df.info() # Displaying column vector of input and output variables

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 1 to 74
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   b_j         74 non-null     float64
 1   h_c         74 non-null     float64
 2   f_c         74 non-null     float64
 3   T_Ast       74 non-null     float64
 4   B_Ast       74 non-null     float64
 5   L           74 non-null     object 
 6   f_yb        74 non-null     float64
 7   phi_s       74 non-null     float64
 8   P_y         74 non-null     float64
 9   x_b         74 non-null     float64
 10  h_b_by_h_c  74 non-null     float64
 11  tau         74 non-null     object 
dtypes: float64(10), object(2)
memory usage: 7.5+ KB


In [95]:
data_df # Displaying original data in pandas data frame

,b_j,h_c,f_c,T_Ast,B_Ast,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c,tau
1,280.0,300.0,31.0,0.8060,0.8060,1500,520.0,0.0000,0.0000,0.1538,1.500,3.75
2,280.0,300.0,33.0,0.8060,0.8060,1500,520.0,0.0819,0.0000,0.1538,1.500,4.631
3,280.0,300.0,42.0,0.8060,0.8060,1500,520.0,0.1442,0.0000,0.1538,1.500,5.7024
4,280.0,300.0,36.0,0.6045,0.6045,1500,520.0,0.0000,0.0000,0.1154,2.000,3.381
5,280.0,300.0,42.0,0.6045,0.6045,1500,520.0,0.0928,0.0000,0.1154,2.000,4.2857
...,...,...,...,...,...,...,...,...,...,...,...,...
70,130.0,150.0,96.0,1.7403,1.7403,840,550.0,0.0000,0.0534,0.2078,1.400,4.8205
71,200.0,300.0,44.0,1.0063,1.0063,1550,594.0,1.3542,0.1500,0.2400,1.333,8.5667
72,200.0,300.0,24.0,1.0063,1.0063,1550,594.0,2.0385,0.1500,0.2400,1.333,6.0667
73,200.0,300.0,25.0,1.0063,1.0063,1550,594.0,1.7996,0.1500,0.2400,1.333,5.45


In [96]:
data_df['L'] = data_df['L'].astype(int)

In [97]:
data_df['tau'] = data_df['tau'].astype(float)

In [98]:
pps_matrix = pps.matrix(data_df) # Applying pps score to pandas data frame, not exactly, but slighly similar to correlation coefficient

In [99]:
pps_score_df = pps_matrix[(pps_matrix['y']=='tau') & (pps_matrix['x']!='tau')] # Calculating pps matrix for x to y with MAE baseline score obtained from navie model and MAE orignal score obtained from Decision Tree Regressor Model
pps_score_df

,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
11,b_j,tau,0.239558,regression,True,mean absolute error,1.433328,1.089964,DecisionTreeRegressor()
23,h_c,tau,0.312111,regression,True,mean absolute error,1.433328,0.985971,DecisionTreeRegressor()
35,f_c,tau,0.403501,regression,True,mean absolute error,1.433328,0.854979,DecisionTreeRegressor()
47,T_Ast,tau,0.493312,regression,True,mean absolute error,1.433328,0.726251,DecisionTreeRegressor()
59,B_Ast,tau,0.493312,regression,True,mean absolute error,1.433328,0.726251,DecisionTreeRegressor()
71,L,tau,0.373446,regression,True,mean absolute error,1.433328,0.898057,DecisionTreeRegressor()
83,f_yb,tau,0.477104,regression,True,mean absolute error,1.433328,0.749481,DecisionTreeRegressor()
95,phi_s,tau,0.186836,regression,True,mean absolute error,1.433328,1.165532,DecisionTreeRegressor()
107,P_y,tau,0.373626,regression,True,mean absolute error,1.433328,0.897799,DecisionTreeRegressor()
119,x_b,tau,0.487390,regression,True,mean absolute error,1.433328,0.734739,DecisionTreeRegressor()


In [125]:
pps_score_df[['x', 'ppscore']].sort_values('ppscore').plot(kind='bar', x='ppscore', y='x') # Plotting pps score on x axis and input features on y axis, thus displaying the importance of various features 

In [ ]:
# X = data_df.drop('tau',axis=1)
# y = data_df['tau']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [101]:
feature_cols = data_df.columns.to_list() # Assigning variable feature_cols to data_df
feature_cols.remove('tau') # Defining feature_cols from data_df by removing tau
feature_cols # Displaying defined variable feature_cols

['b_j',
 'h_c',
 'f_c',
 'T_Ast',
 'B_Ast',
 'L',
 'f_yb',
 'phi_s',
 'P_y',
 'x_b',
 'h_b_by_h_c']

In [102]:
x_mm_sc = MinMaxScaler() # Defining Normalization
y_mm_sc = MinMaxScaler()

In [103]:
X=x_mm_sc.fit_transform(data_df[feature_cols]) # normalizing and transforming into scaled feature variables
y=y_mm_sc.fit_transform(data_df[['tau']]) # normalizing and transforming into scaled output variable tau

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Splitting the entire data into 80% as training set and 20% as testing set and randomlying shulling the data by defining random state of 42

In [104]:
X.shape, y.shape # Recalling the size of input and output varaibles 

((74, 11), (74, 1))

In [105]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape # Recalling the size of train and test set of input and output

((59, 11), (15, 11), (59, 1), (15, 1))

In [106]:
y.shape

(74, 1)

In [126]:
data_df['tau'].hist()

In [127]:
pd.Series(y.reshape(74,)).hist() # Restructuing the output in panda data frame to a single column vector

In [109]:
automl = AutoML() # Assigning the variable automl and calling the fuction AutoML, the algorithms used are lgbm, random forrest, xtremly boosted regression trees, extra tre regressor and xgb limit depth
automl.fit(X_train, y_train, task="regression", metric='mse', time_budget=10) # AutoML function applied to training set, defining task of regression, using evaluation metrics as MSE, and duration of training is defined as 2 hours

[flaml.automl: 12-12 09:05:24] {1943} INFO - task = regression
[flaml.automl: 12-12 09:05:24] {1945} INFO - Data split method: uniform
[flaml.automl: 12-12 09:05:24] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-12 09:05:24] {2019} INFO - Minimizing error metric: mse
[flaml.automl: 12-12 09:05:24] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-12 09:05:24] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-12 09:05:24] {2425} INFO - Estimated sufficient time budget=388s. Estimated necessary time budget=3s.
[flaml.automl: 12-12 09:05:24] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.0549,	best estimator lgbm's best error=0.0549
[flaml.automl: 12-12 09:05:24] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-12 09:05:24] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.0549,	best estimator lgbm's best error=0.0549
[flaml.automl: 12-12 09:05:24] {2311} INFO - i

In [110]:
# Overfitted Model
# xbg_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
#              colsample_bylevel=0.6908489399180993, colsample_bynode=1,
#              colsample_bytree=0.7262491540966294, gamma=0, gpu_id=-1,
#              grow_policy='lossguide', importance_type='gain',
#              interaction_constraints='', learning_rate=0.6222852159633686,
#              max_delta_step=0, max_depth=0, max_leaves=11,
#              min_child_weight=1.85683286845823, missing=np.nan,
#              monotone_constraints='()', n_estimators=19, n_jobs=-1,
#              num_parallel_tree=1, random_state=0,
#              reg_alpha=0.012593125123648567, reg_lambda=0.5275740558733727,
#              scale_pos_weight=1, subsample=0.41724067529198217,
#              tree_method='hist', use_label_encoder=False, validate_parameters=1,
#              verbosity=0)

# Good model
# xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
#              colsample_bylevel=0.7077525003026339, colsample_bynode=1,
#              colsample_bytree=0.6897859838856416, gamma=0, gpu_id=-1,
#              grow_policy='lossguide', importance_type='gain',
#              interaction_constraints='', learning_rate=0.6449648086450813,
#              max_delta_step=0, max_depth=0, max_leaves=11,
#              min_child_weight=1.2743046538759963, missing=np.nan,
#              monotone_constraints='()', n_estimators=21, n_jobs=-1,
#              num_parallel_tree=1, random_state=0, reg_alpha=0.0097635993615457,
#              reg_lambda=0.609620422342749, scale_pos_weight=1,
#              subsample=0.42963574894437784, tree_method='hist',
#              use_label_encoder=False, validate_parameters=1, verbosity=0)


# latest_model

xgb_model = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
             colsample_bylevel=0.9029923859255421, colsample_bynode=1,
             colsample_bytree=0.7002402396813967, gamma=0, gpu_id=0,
            #  gpu_id=-1
             importance_type='gain', interaction_constraints='',
             learning_rate=1.0, max_delta_step=0, max_depth=4,
             min_child_weight=0.1948842518814234, missing=np.nan,
            #  monotone_constraints='()',
              n_estimators=44, n_jobs=-1,
             num_parallel_tree=1, random_state=0,
             reg_alpha=0.0009996619466127016, reg_lambda=15.42416995916816,
             scale_pos_weight=1, subsample=0.8647434526614561,
             tree_method='exact', use_label_encoder=False,
             validate_parameters=1, verbosity=0)

In [111]:
xgb_model.fit(X_train, y_train) # got latest model as xgb with the following hyperparameters and model is trained again on the training set

XGBRegressor(colsample_bylevel=0.9029923859255421,
             colsample_bytree=0.7002402396813967, gpu_id=0,
             interaction_constraints='', learning_rate=1.0, max_depth=4,
             min_child_weight=0.1948842518814234, n_estimators=44, n_jobs=-1,
             num_parallel_tree=1, reg_alpha=0.0009996619466127016,
             reg_lambda=15.42416995916816, subsample=0.8647434526614561,
             tree_method='exact', use_label_encoder=False,
             validate_parameters=1, verbosity=0)

In [112]:
xgb_model.score(X_test, y_test) # Evaluating the R2 score on x test data and y test data

0.8972886017616493

In [113]:
y_test_predicted = xgb_model.predict(X_test) # Assinging variable y_test_predicted to the output of the test data from xgb model

In [114]:
from sklearn.metrics import mean_squared_error # importing MSE function from sklearn library 
mean_squared_error(y_test, y_test_predicted) # MSE for scaled test data for Xgboost model

0.0046871586950080575

In [115]:
y_test_predicted.shape

(15,)

In [116]:
y_test.shape

(15, 1)

In [117]:
actual_test_tau = y_mm_sc.inverse_transform(y_test) # Transforming scaled test data to original fomat
actual_test_tau

array([[4.2857],
       [7.4267],
       [4.85  ],
       [3.75  ],
       [5.7024],
       [6.3333],
       [3.6333],
       [3.6816],
       [3.35  ],
       [6.985 ],
       [2.9744],
       [2.8311],
       [5.55  ],
       [2.9576],
       [4.0714]])

In [118]:
predicted_test_tau = y_mm_sc.inverse_transform(y_test_predicted.reshape(15,1)) # Transforming scaled test data to original fomat
predicted_test_tau

array([[4.3862104],
       [7.1019516],
       [4.405525 ],
       [3.783465 ],
       [6.242198 ],
       [5.763311 ],
       [3.4279556],
       [3.5519915],
       [3.3318453],
       [7.254259 ],
       [4.00141  ],
       [3.1964326],
       [5.8306847],
       [2.4078302],
       [3.2781057]], dtype=float32)

In [119]:
mean_squared_error(actual_test_tau, predicted_test_tau) # MSE of non-scaled test data for Xgboost Model

0.2173913140720506

In [120]:
from sklearn.ensemble import ExtraTreesRegressor

ext_reg_model = ExtraTreesRegressor(max_features=0.7441441386697851, max_leaf_nodes=23, n_estimators=5, n_jobs=-1)

In [121]:
ext_reg_model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



ExtraTreesRegressor(max_features=0.7441441386697851, max_leaf_nodes=23,
                    n_estimators=5, n_jobs=-1)

In [122]:
ext_reg_model.score(X_test, y_test)

0.8822989512322027

In [67]:
y_test_predicted = ext_reg_model.predict(X_test)

In [68]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_test_predicted)  # MSE of Scaled test data for Extra Trees Regression Model

0.005393518407456412

In [69]:
y_test_predicted.shape

(15,)

In [70]:
y_test.shape

(15, 1)

In [71]:
actual_test_tau = y_mm_sc.inverse_transform(y_test) # Transforming actual scaled test data to original fomat
actual_test_tau

array([[4.2857],
       [7.4267],
       [4.85  ],
       [3.75  ],
       [5.7024],
       [6.3333],
       [3.6333],
       [3.6816],
       [3.35  ],
       [6.985 ],
       [2.9744],
       [2.8311],
       [5.55  ],
       [2.9576],
       [4.0714]])

In [72]:
predicted_test_tau = y_mm_sc.inverse_transform(y_test_predicted.reshape(15,1)) # Transforming predicted scaled test data to original fomat
predicted_test_tau

array([[3.80868   ],
       [6.9536775 ],
       [4.27666667],
       [4.0858613 ],
       [5.12333273],
       [5.82279667],
       [3.39029   ],
       [3.92645   ],
       [3.58945667],
       [7.31108893],
       [4.04651   ],
       [3.18472857],
       [5.29101939],
       [2.62475   ],
       [3.3444    ]])

In [73]:
mean_squared_error(actual_test_tau, predicted_test_tau) # MSE of non-scaled test data

0.2501523874176688

In [74]:
math.sqrt(mean_squared_error(actual_test_tau, predicted_test_tau)) # RMSE of non-scaled test data

0.5001523642028185

In [75]:
y_train_predicted = xgb_model.predict(X_train)

In [76]:
actual_train_tau = y_mm_sc.inverse_transform(y_train) # Transforming actual scaled train data to original fomat
actual_train_tau

array([[6.012 ],
       [8.2186],
       [4.7636],
       [3.4612],
       [2.925 ],
       [8.5667],
       [3.6   ],
       [6.4157],
       [3.3455],
       [8.5103],
       [4.1   ],
       [8.2269],
       [2.3273],
       [4.0595],
       [3.6   ],
       [4.6545],
       [4.4   ],
       [7.7935],
       [4.75  ],
       [2.875 ],
       [3.65  ],
       [6.3394],
       [3.381 ],
       [4.2   ],
       [6.686 ],
       [5.8   ],
       [6.4098],
       [4.3   ],
       [7.2844],
       [2.9744],
       [7.1767],
       [3.1515],
       [5.45  ],
       [3.6   ],
       [4.631 ],
       [3.1   ],
       [4.7976],
       [4.1939],
       [3.381 ],
       [7.3184],
       [4.359 ],
       [3.45  ],
       [2.832 ],
       [4.8205],
       [8.6103],
       [6.9498],
       [3.7619],
       [2.7152],
       [3.1795],
       [4.631 ],
       [5.8061],
       [1.9   ],
       [5.7024],
       [3.75  ],
       [1.8   ],
       [8.0685],
       [6.0667],
       [3.3   ],
       [4.4485

In [77]:
predicted_train_tau = y_mm_sc.inverse_transform(y_train_predicted.reshape(59,1)) # Transforming predicted scaled train data to original fomat
predicted_train_tau

array([[5.9982243],
       [8.186414 ],
       [4.8682504],
       [3.4760501],
       [2.8750978],
       [8.548995 ],
       [3.5911486],
       [6.4404984],
       [3.322113 ],
       [8.390867 ],
       [4.0999913],
       [8.267725 ],
       [2.4078302],
       [3.8824666],
       [3.509756 ],
       [4.6929955],
       [4.405525 ],
       [7.694031 ],
       [4.7493563],
       [2.9010131],
       [3.6403074],
       [6.2130175],
       [3.3083167],
       [4.192933 ],
       [6.6884866],
       [5.7916684],
       [6.7757974],
       [4.2708   ],
       [7.242376 ],
       [3.1261377],
       [7.2543306],
       [3.0989695],
       [5.5857406],
       [3.5911486],
       [4.6418386],
       [3.1128333],
       [4.761462 ],
       [4.105362 ],
       [3.4155848],
       [7.2485557],
       [4.22173  ],
       [3.4279556],
       [2.8416014],
       [4.8930035],
       [8.524208 ],
       [6.949275 ],
       [3.8824666],
       [2.7675385],
       [3.1437345],
       [4.6796966],


In [78]:
mean_squared_error(actual_train_tau, predicted_train_tau) # MSE of non-scaled train data

0.007618598894801782

In [79]:
math.sqrt(mean_squared_error(actual_train_tau, predicted_train_tau))  # RMSE of non-scaled train data

0.08728458566552162

In [80]:
x_train_df = pd.DataFrame(x_mm_sc.inverse_transform(X_train), columns = feature_cols)
x_train_df.head()

,b_j,h_c,f_c,T_Ast,B_Ast,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c
0,280.0,300.0,34.0,1.2090,1.2090,1500.0,520.0,0.0000,0.1607,0.2308,1.000
1,337.0,356.0,56.0,1.6361,1.6361,1600.0,458.0,1.3810,0.0740,0.2556,1.427
2,275.0,300.0,43.0,0.7856,0.7856,850.0,485.0,0.2005,0.1691,0.1200,1.667
3,350.0,350.0,24.0,0.7272,0.7272,1780.0,417.0,1.5629,0.0000,0.1800,1.140
4,200.0,200.0,16.0,0.7700,0.7700,1100.0,585.0,0.0000,0.0719,0.1400,1.500


In [81]:
len(x_train_df)

59

In [82]:
y_train_df = pd.DataFrame(y_mm_sc.inverse_transform(y_train), columns = ['tau'])
y_train_df.head()

,tau
0,6.0120
1,8.2186
2,4.7636
3,3.4612
4,2.9250


In [83]:
len(y_train_df)

59

In [84]:
train_df = x_train_df.join(y_train_df)

In [85]:
predicted_train_df = pd.DataFrame(predicted_train_tau, columns=['predicted_tau'])
predicted_train_df.head()

,predicted_tau
0,5.998224
1,8.186414
2,4.868250
3,3.476050
4,2.875098


In [ ]:
len(predicted_train_df)

59

In [ ]:
train_df = train_df.join(predicted_train_df)

In [ ]:
train_df.head()

,b_j,h_c,f_c,T_Ast,B_Ast,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c,tau,predicted_tau
0,280.0,300.0,34.0,1.2090,1.2090,1500.0,520.0,0.0000,0.1607,0.2308,1.000,6.0120,6.023890
1,337.0,356.0,56.0,1.6361,1.6361,1600.0,458.0,1.3810,0.0740,0.2556,1.427,8.2186,8.160102
2,275.0,300.0,43.0,0.7856,0.7856,850.0,485.0,0.2005,0.1691,0.1200,1.667,4.7636,4.835969
3,350.0,350.0,24.0,0.7272,0.7272,1780.0,417.0,1.5629,0.0000,0.1800,1.140,3.4612,3.456226
4,200.0,200.0,16.0,0.7700,0.7700,1100.0,585.0,0.0000,0.0719,0.1400,1.500,2.9250,2.941785


In [ ]:
x_test_df = pd.DataFrame(x_mm_sc.inverse_transform(X_test), columns = feature_cols)
x_test_df.head()

,b_j,h_c,f_c,T_Ast,B_Ast,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c
0,280.0,300.0,42.0,0.6045,0.6045,1500.0,520.0,0.0928,0.0000,0.1154,2.000
1,337.0,356.0,94.0,1.2158,1.2158,1600.0,458.0,0.8915,0.0211,0.2204,1.427
2,100.0,200.0,23.0,1.1800,1.1800,1000.0,470.0,0.2450,0.1000,0.3000,1.000
3,280.0,300.0,31.0,0.8060,0.8060,1500.0,520.0,0.0000,0.0000,0.1538,1.500
4,280.0,300.0,42.0,0.8060,0.8060,1500.0,520.0,0.1442,0.1607,0.1538,1.500


In [ ]:
len(x_test_df)

15

In [ ]:
y_test_df = pd.DataFrame(y_mm_sc.inverse_transform(y_test), columns = ['tau'])
y_test_df.head()

,tau
0,4.2857
1,7.4267
2,4.8500
3,3.7500
4,5.7024


In [ ]:
len(y_test_df)

15

In [ ]:
test_df = x_test_df.join(y_test_df)

In [ ]:
predicted_test_df = pd.DataFrame(predicted_test_tau, columns=['predicted_tau'])
predicted_test_df.head()

,predicted_tau
0,4.076765
1,7.093825
2,4.380329
3,4.183898
4,5.859646


In [ ]:
test_df = test_df.join(predicted_test_df)

In [ ]:
test_df.head()

,b_j,h_c,f_c,T_Ast,B_Ast,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c,tau,predicted_tau
0,280.0,300.0,42.0,0.6045,0.6045,1500.0,520.0,0.0928,0.0000,0.1154,2.000,4.2857,4.076765
1,337.0,356.0,94.0,1.2158,1.2158,1600.0,458.0,0.8915,0.0211,0.2204,1.427,7.4267,7.093825
2,100.0,200.0,23.0,1.1800,1.1800,1000.0,470.0,0.2450,0.1000,0.3000,1.000,4.8500,4.380329
3,280.0,300.0,31.0,0.8060,0.8060,1500.0,520.0,0.0000,0.0000,0.1538,1.500,3.7500,4.183898
4,280.0,300.0,42.0,0.8060,0.8060,1500.0,520.0,0.1442,0.1607,0.1538,1.500,5.7024,5.859646


In [ ]:
train_df.round(4, inplace=True)

,b_j,h_c,f_c,T_Ast,B_Ast,L,f_yb,phi_s,P_y,x_b,h_b_by_h_c,tau,predicted_tau
0,280.0,300.0,34.0,1.2090,1.2090,1500.0,520.0,0.0000,0.1607,0.2308,1.000,6.0120,6.0239
1,337.0,356.0,56.0,1.6361,1.6361,1600.0,458.0,1.3810,0.0740,0.2556,1.427,8.2186,8.1601
2,275.0,300.0,43.0,0.7856,0.7856,850.0,485.0,0.2005,0.1691,0.1200,1.667,4.7636,4.8360
3,350.0,350.0,24.0,0.7272,0.7272,1780.0,417.0,1.5629,0.0000,0.1800,1.140,3.4612,3.4562
4,200.0,200.0,16.0,0.7700,0.7700,1100.0,585.0,0.0000,0.0719,0.1400,1.500,2.9250,2.9418
5,200.0,300.0,44.0,1.0063,1.0063,1550.0,594.0,1.3542,0.1500,0.2400,1.333,8.5667,8.5386
6,100.0,200.0,21.0,0.7850,0.7850,1000.0,470.0,0.5179,0.1000,0.2000,1.000,3.6000,3.6100
7,370.0,420.0,67.0,1.3639,1.3639,1900.0,430.0,1.0651,0.0188,0.2917,1.071,6.4157,6.4170
8,275.0,300.0,37.0,0.7856,0.7856,850.0,570.0,0.0000,0.1867,0.1200,1.667,3.3455,3.3299
9,337.0,356.0,74.0,1.6361,1.6361,1600.0,458.0,1.2014,0.0634,0.2556,1.427,8.5103,8.4854


In [ ]:
train_df.to_csv('data/training_data_w_pred_values.csv', index=False)

In [ ]:
test_df.to_csv('data/test_data_w_pred_values.csv', index=False)